In [4]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 17.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.8/322.8 KB 16.7 MB/s eta 0:00:00


In [5]:
import numpy as np
import plotly.graph_objects as go

# Base Map Overlay visualizer

In [ ]:
# Define your waypoints (lat, lon)
waypoints = [
    (49.2827, -123.1207),  # Vancouver
    (48.4284, -123.3656),  # Victoria
    (47.6062, -122.3321),  # Seattle
    (45.5122, -122.6587)   # Portland
]

# Interpolate between waypoints
def interpolate_waypoints(points, steps=50):
    path = []
    for i in range(len(points) - 1):
        lat0, lon0 = points[i]
        lat1, lon1 = points[i + 1]
        lats = np.linspace(lat0, lat1, steps)
        lons = np.linspace(lon0, lon1, steps)
        path.extend(zip(lats, lons))
    return path

# Generate interpolated path
boat_path = interpolate_waypoints(waypoints, steps=50)

# Create the base frame with waypoints
waypoint_trace = go.Scattergeo(
    lon=[lon for lat, lon in waypoints],
    lat=[lat for lat, lon in waypoints],
    mode='markers+text',
    marker=dict(size=8, color='red'),
    text=["Vancouver", "Victoria", "Seattle", "Portland"],
    textposition='top center',
    name='Waypoints'
)

# Initial boat marker
boat_trace = go.Scattergeo(
    lon=[boat_path[0][1]],
    lat=[boat_path[0][0]],
    mode='markers',
    marker=dict(size=12, color='blue', symbol='triangle-up'),
    name='Boat'
)

# Create animation frames
frames = [
    go.Frame(
        data=[
            go.Scattergeo(
                lon=[lon],
                lat=[lat],
                mode='markers',
                marker=dict(size=12, color='blue', symbol='triangle-up'),
                name='Boat'
            )
        ],
        name=str(i)
    )
    for i, (lat, lon) in enumerate(boat_path)
]

# Create figure
fig = go.Figure(
    data=[waypoint_trace, boat_trace],
    frames=frames,
    layout=go.Layout(
        title="Interactive Boat Journey",
        geo=dict(
            scope='north america',
            projection_type='mercator',
            showland=True,
            landcolor='lightgray',
            showocean=True,
            oceancolor='lightblue',
        ),
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            buttons=[
                dict(
                    label='Play',
                    method='animate',
                    args=[None, {
                        'frame': {'duration': 50, 'redraw': True},
                        'fromcurrent': True,
                        'transition': {'duration': 0}
                    }]
                )
            ]
        )]
    )
)

fig.show()